In [1]:
import pandas as pd 
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# create a config class to store colors and font for all figures 
class config: 
    colors = [ '#387da8', '#cf514e']
    font = 'Arial'
    dem = 'rgb(0, 174, 243)'
    rep = 'rgb(232, 27, 35)'

In [6]:
# Figure 2: Opposition to COVID-19 Policies by Political Party

# read dataset 
df = pd.read_csv('analysis_data.csv')

# convert date to datetime
df.date = pd.to_datetime(df.date)

# make the two independent senators democrats since they caucus with the democrats
df.party = df.party.replace({'Independent': 'Democrat'})

# create an opposition percentage column 
df['opposition'] = df['opposition_tweet_count'] / df.total_tweets

# group the dataset by party and date and calculate the mean and standard error of the opposition
party = df.groupby(['party', 'date']).agg({'opposition': ['mean', 'sem']}).reset_index()

# rename columns
party.columns = ['party', 'date', 'opposition', 'sem']

    
# create the plot
fig = px.line(party, x='date', y='opposition', color='party', color_discrete_sequence=[config.dem, config.rep],
              labels={'date':'Date', 'opposition':'Opposition'}, 
              title='Opposition to COVID-19 Policies by Party', 
              )


for p in party.party.unique():
    x1 = party.loc[party.party == p]
    if p == 'Democrat':
        color = 'rgba(0,176,246,0.2)'
    if p == 'Republican':
        color = 'rgba(255,0,0,0.2)'
    
    fig.add_trace(go.Scatter(
        x=x1.date,
        y=x1['opposition'] + x1['sem'],
        mode='lines',
        name='Upper Bound',
        line=dict(color='rgba(0,0,0,0)'),
        showlegend=False
    ))

    fig.add_trace(go.Scatter(
        x=x1.date,
        y=x1['opposition'] - x1['sem'],
        mode='lines',
        name='Lower Bound',
        line=dict(color='rgba(0,0,0,0)'),
        fill='tonexty',
        fillcolor=color,
        showlegend=False
    ))



fig.update_layout(
    font_family="Arial",
    font_color="black",
    title_font_family="Arial",
    title_font_color="black",
    title_font_size = 16,
    font_size = 16,
    title = '<b>Expressed Opposition to COVID-19 Policies by<br>US Congress Members on Twitter</b>',
    legend_title_font_color="black", 
    title_y=0.95,
    template='presentation',
    yaxis_title="Opposition",
    yaxis_tickformat='2%',
    xaxis_title="",
    xaxis_tickformat='%b-%y',
    title_x=0.05, 
    showlegend=True,
    width=800,
    height=500,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=.95,
        font=dict(size=18, color="black", family=config.font),
    ), 
    legend_title_text='',
)


# reduce margins 
fig.update_layout(margin=dict(l=70, r=40, t=100, b=40, pad=1))
# make lines thicker
fig.update_traces(line=dict(width=4))


fig.show()
#fig.write_image("expressed_opposition_twitter.png")


In [4]:
pip install -U kaleido

Note: you may need to restart the kernel to use updated packages.
